# Compute opacities

In [ ]:
import warnings

import matplotlib.pyplot as plt
import numpy as np
import optool
from tqdm.auto import tqdm

from helper import Capturing, apply_araa_style

In [ ]:
settings = {}
settings['DSHARP'] = 'astrosil 0.3291 c-org 0.3966 fes 0.0743 h2o-w 0.2000 -mie'
settings['DSHARP-ZBED'] = 'astrosil 0.3291 c 0.3966 fes 0.0743 h2o-w 0.2000 -dhs'
settings['DIANA'] = '-dhs'

Note that there were issues with optool that magically disappeared. here's the test in case it happens again. The first entry of fp12 should be the same and the rest not different by many orders of magnitude.
```python
p1 = optool.particle(f'optool -mie -dsharp -a 1.0 1000.0 3.5 100 -l 1250 -s 180')
p2 = optool.particle(f'optool -mie -dsharp -a 1.0 1000.0 3.5 100 -l 1250 3000 5 -s 180')

for p in [p1, p2]:
    print(f'lam = {p.lam}')
    print(f'fp12 = {p.f12[0, :, 89]}')
```

In [ ]:
na = 200
a = np.geomspace(1e-3, 1e2, na)
amin = 1.e-5
lam = [0.125, 0.3]
nlam = len(lam)
q = 3.5

res = {}
out = Capturing()

for key in settings.keys():

    d = {}
    res[key] = d

    d['lam']   = lam
    d['amax']  = a
    d['k_abs'] = np.zeros([na, nlam])
    d['k_sca'] = np.zeros([na, nlam])
    d['omega'] = np.zeros([na, nlam])
    d['Pscat'] = np.zeros([na, nlam])
    
    for i, amax in tqdm(enumerate(a), total=na):
        p = optool.particle(f'optool {settings[key]} -a {amin * 1e4} {amax * 1e4} {q} {10 + 5 * i} -l {1e4 * lam[0]} {1e4 * lam[-1]} {nlam} -s',
                            silent=True)
        
        # supress the warning in the conversion
        with out:
            p.scatnorm('r')

        i895 = np.abs(p.scatang-89.5).argmin()
        if not np.isclose(p.scatang[i895], 89.5) or (not np.isclose(p.scatang[i895+1], 90.5)):
            warnings.warn('scattering angles not exactly at 89.5º and 90.5º' )
            
        d['k_abs'][i, :] = p.kabs
        d['k_sca'][i, :] = p.ksca
        d['omega'][i, :] = p.ksca / p.kext
        d['Pscat'][i, :] = np.abs(p.f12[0, :, i895:i895+2] / p.f11[0, :, i895:i895+2]).mean(-1)

# Plot

In [ ]:
ilam = 0

f, axs = plt.subplots(3, 1, sharex=True, figsize=(4, 9), gridspec_kw={'hspace':0.05})


##### plot the kappas (mass absorption/scattering coefficients) #######
ax = axs[0]

for i,key in enumerate(settings.keys()):
    line1, = ax.loglog(a, res[key]['k_abs'][:,ilam], label=key)
    line2, = ax.loglog(a, res[key]['k_sca'][:,ilam], '--', c=line1.get_color(), alpha=0.5)

ax.plot([],[], 'k', ls=line1.get_linestyle(), label=r'$\bar\kappa_{abs}$')
ax.plot([],[], 'k', ls=line2.get_linestyle(), label=r'$\bar\kappa_{abs}$', alpha=line2.get_alpha())
ax.legend(fontsize='small', frameon=False)

ax.set_ylim(2e-2, 50)
ax.set_ylabel(r'$\bar \kappa$ [cm$^2$/g]')

ax.set_title(f'$q = {q}$')


####### plot the spectral index beta #######
ax = axs[1]
for key in settings.keys():
    beta = -np.log(res[key]['k_abs'][:, 1] / res[key]['k_abs'][:, 0])/np.log(res[key]['lam'][1] / res[key]['lam'][0])
    ax.semilogx(a, beta, '-', label=key)
    
ax.set_ylim(0,3.5)
ax.set_ylabel(r'opacity spectral index $\beta$')


#### Albedo & 90º scattering probability #####
ax = axs[2]

for key in settings.keys():
    line1, = ax.semilogx(a, res[key]['omega'][:, ilam], '--', alpha=0.5)
    line2, = ax.semilogx(a, res[key]['Pscat'][:, ilam], ':', c=line1.get_color(), alpha=0.5)
    line3, = ax.semilogx(a, res[key]['Pscat'][:, ilam] * res[key]['omega'][:, ilam], '-', c=line1.get_color())

ax.plot([], [], c='k', ls=line1.get_linestyle(), alpha=line1.get_alpha(), label='$\omega$')
ax.plot([], [], c='k', ls=line2.get_linestyle(), alpha=line2.get_alpha(), label='$P_{90}$')
ax.plot([], [], c='k', ls=line3.get_linestyle(), alpha=line3.get_alpha(), label='$\omega\cdot P_{90}$')

for i,key in enumerate(settings.keys()):
    line3, = ax.semilogx(a, res[key]['Pscat'][:, ilam+1] * res[key]['omega'][:, ilam+1], '-', c='k', alpha=0.25)
    
ax.set_xlim(a[[0, -1]])
ax.set_ylim(0,1.1)
ax.legend()
ax.set_xlabel('maximum particle size [cm]')
ax.set_ylabel(r'$\omega, P_{90}, \omega \cdot P_{90}$')

for ax in axs.ravel():
    apply_araa_style(ax)

f.savefig(f'opacity_plot_q{str(q).replace(".", "")}.pdf')

## compute the `npz` file for DSHARP-ZD

In [ ]:
from tqdm.auto import tqdm
import numpy as np
import optool

key = 'DSHARP-ZBED'

# define the size and wavelength grids

lmin = 1e-5
lmax = 1e2
nlam = 210
na   = 200

a = np.geomspace(1e-5, 1e2, na)
lam = np.geomspace(lmin, lmax, nlam)

# compute the opacities, size by size
k_a = []
k_s = []
g_s = []
for size in tqdm(a):
    p = optool.particle(f'optool {settings[key]} -l {lmin * 1e4} {lmax * 1e4} {nlam} -a {size * 1e4}', silent=True)
    k_a.append(p.kabs)
    k_s.append(p.ksca)
    g_s.append(p.gsca)
    
k_abs = np.vstack(k_a)
k_sca = np.vstack(k_s)
g_sca = np.vstack(g_s)

# Saving to file

np.savez(
    f"{key.replace('-', '_')}.npz",
    a=a,               # Size grid
    lam=lam,           # Wavelength grid
    k_abs=k_abs,       # Absorption opacity
    k_sca=k_sca,       # Scattering opacity
    g=g_sca,           # Asymmetry factor
    rho_s=p.rho,       # Material density
    composition=settings[key] # Parameter dictionary
)

# Compare optool and sharp Zubko lnk data

In [ ]:
dzb_o = do.diel_from_lnk_file('/Users/birnstiel/CODES/optool/lnk_data/c-z-Zubko1996.lnk', headerlines=12)
do.compare_nk([dzb_d, dz_o])

# Generate the `ACH2` lnk-file in optool-forma

In [ ]:
import dsharp_opac as do

dzb_d = do.diel_zubko96(sample='BE')
dza_d = do.diel_zubko96(sample='ACH2')
lnk_z_ACH2 = np.array([dza_d._l * 1e4, dza_d._n, dza_d._k]).T

header = \
"""# Material:  Carbon, amorphous
# Reference: Zubko 1996, MNRAS 282, 1321
#
# Name:       Carbon
# Class:      Carbon-ACH2
# State:      amorphous
# Formula:    C
# ADS-link:   https://ui.adsabs.harvard.edu/abs/1996MNRAS.282.1321Z
# BibTeX-key: 1996MNRAS.282.1321Z
#
# First line is N_lam and rho [g/cm^3], other lines are: lambda[um] n k
"""

with open('c-zach2-Zubko1996.lnk', 'w') as fid:
    fid.write(header)
    fid.write(f'{len(lnk_z_ACH2)} nan\n')
    np.savetxt(fid, lnk_z_ACH2, fmt='%0.5E', delimiter='\t')